# Structured Output with Anthropic
In order to build reliable pipelines in which LLMs consistently return output in the same format, we are using a **Structured Output**
- This means that we define a blueprint for the output
- We pass the 'blueprint' to the LLM
- Then the LLM output will conform to the blueprint.

This 'blueprint' in the LLM jargon is often called a "schema".

**To generate structured outputs with Anthropic, we'll use the library `instructor`**

In [ ]:
!pip install anthropic instructor --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.2 MB/s eta 0:00:00


In [ ]:
import anthropic
from google.colab import userdata
from pydantic import BaseModel, Field
from typing import List, Optional
import instructor

ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

We use pydantic to create data models.
- Here we would like to imagine cities for a role-playing-game scenarios.

In [ ]:
class City(BaseModel):
    name: str = Field(..., description="Name of the city. Can be inspired from Oriental, Asian, Russian, Amercian, European styles")
    biotope: str = Field(..., description="The natural surroundings or environmental features of the city")
    economy: str = Field(..., description="The primary industries, economic status, or key exports/imports")
    culture: str = Field(..., description="Traditions, festivals, social norms, and attitudes towards outsiders")
    military: Optional[str] = Field(None, description="Details about the city's defense systems, military force, or recent conflicts")
    technology: Optional[str] = Field(None, description="Level of technological or magical advancement, and its accessibility")
    notable_features: Optional[List[str]] = Field(None, description="List of unique landmarks, key buildings, or hidden locations")
    population: Optional[str] = Field(None, description="Demographics, dominant species, and population size")
    mood: Optional[str] = Field(None, description="General atmosphere, sensory details, or challenges faced by residents")

We can inject some initial ideas we might have

In [ ]:
idea_seeds = [
    'A city name Aquabah',
    'A city named Kniga where human built mechanical creatures and machine to assist them',
    'something with Italian/German vibes',
    'Very cold place',
    'Desert'
]

Note below how:
- we pass the `antropic_client` to **`instructor`**
- we loop through our initial ideas
- inject ideas into the user prompt template
- have high temperature for good creativity
- pass the **`City`** pydantic data schema to the `response_model` argument.

In [ ]:
antropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
client = instructor.from_anthropic(antropic_client)

cities = []
for idea in idea_seeds:
    city = client.messages.create(
        model="claude-3-5-haiku-latest",
        max_tokens=2056,
        messages=[
            {
                "role": "system",
                "content": "You are a helpful table-top RPG gamemaster assistant, \
                and a world class scenario writer."},
            {
                "role": "user",
                "content": f"Imagine a city on the basis of the following idea: {idea}. \
                 Be creative. \
                 Provide only one sentence per attribute of the city. Be very concise. \
                 Use language adequat for teenagers."
            },
        ],
        temperature=0.9,
        response_model=City,
    )
    cities.append(city)

Now we can see the 5 `cities` objects which have been created.

In [ ]:
len(cities)

5

In [ ]:
from pprint import pprint

In [ ]:
print(cities[3].name, '\n')
pprint(cities[3].model_dump())

Frostholm 

{'biotope': 'A desolate arctic landscape with endless snow-covered plains, '
            'jagged glaciers, and minimal vegetation surviving in the harsh, '
            'freezing environment',
 'culture': 'Tough, close-knit community where survival skills are more '
            'important than social status, and everyone learns to adapt or '
            'perish in the extreme cold',
 'economy': 'Survival-based economy centered on ice mining, rare mineral '
            'extraction, and advanced thermal technology trade',
 'military': 'Elite cold-warfare trained defense force with specialized arctic '
             'combat gear and mobility vehicles',
 'mood': 'A constant sense of grim determination mixed with an underlying '
         'excitement of living on the edge of human survival',
 'name': 'Frostholm',
 'notable_features': ['Obsidian Ice Citadel',
                      'Thermal Energy Reactor',
                      'Underground Heated Tunnels'],
 'population': 'Mostly h

Now what? **We could use those generate cities and generate a couple of NPCs for each city** 😱
As you can see, structured output are powerful!

Learn more about instructor here: https://python.useinstructor.com/